## Train Cartoon GAN

Jupyter notebook version of the ```src.networks.train.py``` file.
This file contains all cells to train the model from scratch.

#### Configure drive (only if needed)

In [ ]:
"""
from google.colab import drive
import os
drive.mount("/content/drive")

PROJECT_DIRECTORY = "drive/MyDrive/DeepL"
os.chdir(PROJECT_DIRECTORY)

!ls
# !pip install -r requirements.txt
"""

In [1]:
import torch
import torch.optim as optim


cuda = torch.cuda.is_available()
print(cuda)

if cuda:
  print(torch.cuda.get_device_name(0))
  !nvidia-smi

device = "cuda" if cuda else "cpu"

False


#### Main parameters

It is easier to change some parameters in `.ipynb` file instead of `config.py` while using Google colaboratory.

In [2]:
RESULTS_SAVE_PATH = ""
epochs_train = 50
epochs_pretraining = 50
lr_generator = 0.001
lr_discriminator = 0.001


#### Prepare data

In [ ]:
from src.dataset.cartoon_loader import CartoonDatasetLoader
from src.dataset.pictures_loader import PicturesDatasetLoader

#### Initialize networks

In [ ]:
from time import time
import src.networks as networks

In [ ]:
generator = networks.Generator()
discriminator = networks.Discriminator()
vgg19 = networks.VGG19()

generator.to(device)
discriminator.to(device)
vgg19.to(device)

# to train mode
generator.train()
discriminator.train()
# no training
vgg19.eval()

print("------ Networks -------")
networks.print_network(generator)
networks.print_network(discriminator)
networks.print_network(vgg19)
print("-----------------------")

In [ ]:
G_optimizer = optim.Adam(
    generator.parameters(), 
    lr=lr_generator, 
    betas=(args.beta1, args.beta2)
)
D_optimizer = optim.Adam(
    discriminator.parameters(), 
    lr=lr_discriminator, 
    betas=(args.beta1, args.beta2)
)
G_scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer=G_optimizer, 
    milestones=[epochs // 2, epochs // 4 * 3],
    gamma=0.1
)
D_scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer=D_optimizer, 
    milestones=[epochs // 2, epochs // 4 * 3],
    gamma=0.1
)

### Pretraining

In [ ]:
pre_train_hist = {}
pre_train_hist['Recon_loss'] = []
pre_train_hist['per_epoch_time'] = []
pre_train_hist['total_time'] = []

In [ ]:
start_time = time()

for epoch in range(pre_train_epoch):

    epoch_start_time = time()
    Recon_losses = []
    for x, _ in train_loader_src:
        x = x.to(device)

        # train generator G
        G_optimizer.zero_grad()

        x_feature = VGG((x + 1) / 2)
        G_ = G(x)
        G_feature = VGG((G_ + 1) / 2)

        Recon_loss = 10 * L1_loss(G_feature, x_feature.detach())
        Recon_losses.append(Recon_loss.item())
        pre_train_hist['Recon_loss'].append(Recon_loss.item())

        Recon_loss.backward()
        G_optimizer.step()

    per_epoch_time = time.time() - epoch_start_time
    pre_train_hist['per_epoch_time'].append(per_epoch_time)
    print('[%d/%d] - time: %.2f, Recon loss: %.3f' % ((epoch + 1), args.pre_train_epoch, per_epoch_time, torch.mean(torch.FloatTensor(Recon_losses))))

total_time = time.time() - start_time
pre_train_hist['total_time'].append(total_time)


with open(os.path.join(args.name + '_results',  'pre_train_hist.pkl'), 'wb') as f:
    pickle.dump(pre_train_hist, f)

with torch.no_grad():
    G.eval()
    for n, (x, _) in enumerate(train_loader_src):
        x = x.to(device)
        G_recon = G(x)
        result = torch.cat((x[0], G_recon[0]), 2)
        path = os.path.join(args.name + '_results', 'Reconstruction', args.name + '_train_recon_' + str(n + 1) + '.png')
        plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
        if n == 4:
            break

    for n, (x, _) in enumerate(test_loader_src):
        x = x.to(device)
        G_recon = G(x)
        result = torch.cat((x[0], G_recon[0]), 2)
        path = os.path.join(args.name + '_results', 'Reconstruction', args.name + '_test_recon_' + str(n + 1) + '.png')
        plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
        if n == 4:
            break